In [1]:
import sqlite3
conn = sqlite3.connect('mimic_iv_hosp.db')
cursor = conn.cursor()

In [2]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()
for table in tables:
    print(table[0])

admissions.csv
diagnoses_icd.csv
drgcodes.csv
d_hcpcs.csv
d_icd_diagnoses.csv
d_icd_procedures.csv
d_labitems.csv
emar.csv
hcpcsevents.csv
microbiologyevents.csv
omr.csv
patients.csv
pharmacy.csv
poe.csv
poe_detail.csv
prescriptions.csv
procedures_icd.csv
provider.csv
services.csv
transfers.csv
emar_detail.csv
labevents.csv


In [3]:
cursor.execute('PRAGMA table_info("admissions.csv")')
for row in cursor.fetchall():
    print(row)

(0, 'subject_id', 'INTEGER', 0, None, 0)
(1, 'hadm_id', 'INTEGER', 0, None, 0)
(2, 'admittime', 'TEXT', 0, None, 0)
(3, 'dischtime', 'TEXT', 0, None, 0)
(4, 'deathtime', 'TEXT', 0, None, 0)
(5, 'admission_type', 'TEXT', 0, None, 0)
(6, 'admit_provider_id', 'TEXT', 0, None, 0)
(7, 'admission_location', 'TEXT', 0, None, 0)
(8, 'discharge_location', 'TEXT', 0, None, 0)
(9, 'insurance', 'TEXT', 0, None, 0)
(10, 'language', 'TEXT', 0, None, 0)
(11, 'marital_status', 'TEXT', 0, None, 0)
(12, 'race', 'TEXT', 0, None, 0)
(13, 'edregtime', 'TEXT', 0, None, 0)
(14, 'edouttime', 'TEXT', 0, None, 0)
(15, 'hospital_expire_flag', 'INTEGER', 0, None, 0)


In [4]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = [t[0] for t in cursor.fetchall()]

In [5]:
key_fields = {"subject_id", "hadm_id", "stay_id", "icd_code"}

In [6]:
schema_info = []

for table in table_names:
    cursor.execute(f'PRAGMA table_info("{table}")')
    for row in cursor.fetchall():
        column_name = row[1]
        schema_info.append({
            "table": table,
            "column": column_name,
            "type": row[2],
        })

In [7]:
import pandas as pd
df = pd.DataFrame(schema_info)

In [8]:
print(df)

              table           column     type
0    admissions.csv       subject_id  INTEGER
1    admissions.csv          hadm_id  INTEGER
2    admissions.csv        admittime     TEXT
3    admissions.csv        dischtime     TEXT
4    admissions.csv        deathtime     TEXT
..              ...              ...      ...
224   labevents.csv  ref_range_lower     REAL
225   labevents.csv  ref_range_upper     REAL
226   labevents.csv             flag     TEXT
227   labevents.csv         priority     TEXT
228   labevents.csv         comments     TEXT

[229 rows x 3 columns]


In [10]:
admissions = pd.read_sql_query('SELECT * FROM "admissions.csv"', conn)
diagnoses = pd.read_sql_query('SELECT * FROM "diagnoses_icd.csv"', conn)
diag_defs = pd.read_sql_query('SELECT * FROM "d_icd_diagnoses.csv"', conn)
labs = pd.read_sql('SELECT * FROM "labevents.csv" LIMIT 1000000', conn)       # sample for speed
lab_defs = pd.read_sql('SELECT * FROM "d_labitems.csv"', conn)
presc = pd.read_sql('SELECT * FROM "prescriptions.csv"', conn)

: 

: 

In [ ]:
lab

In [ ]:
diagnoses

In [ ]:
diag_defs

merging diagnoses with descriptions

In [ ]:
diagnoses = diagnoses.merge(
    diag_defs,
    on=["icd_code", "icd_version"],
    how="left"
)

Grouping diagnoses by admission

In [ ]:
grouped_diags = diagnoses.groupby("hadm_id")["long_title"].apply(lambda x: "; ".join(x.dropna().unique())).reset_index()
grouped_diags.rename(columns={"long_title": "diagnosis_summary"}, inplace=True)

Merge into admissions

In [ ]:
enriched = admissions.merge(grouped_diags, on="hadm_id", how="left")

In [ ]:
enriched

In [ ]:
def build_summary(row):
    outcome = "Died in hospital" if row.hospital else "Discharged alive"
    return f"""Patient ID: {row.subject_id}
Admission ID: {row.hadm_id}
Admission Type: {row.admission_type}
Admission Time: {row.admittime}
Discharge Time: {row.dischtime}
Insurance: {row.insurance}
Marital Status: {row.marital_status}
Race: {row.race}
Diagnoses: {row.diagnosis_summary if pd.notna(row.diagnosis_summary) else "N/A"}"""

enriched["summary"] = enriched.apply(build_summary, axis=1)

In [ ]:
for text in enriched["summary"].head(5):
    print(text)
    print("-" * 40)

In [ ]:
enriched

In [ ]:
sampled = enriched.sample(5000, random_state=80)

In [ ]:
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS


documents = [
    Document(
        page_content=row["summary"],
        metadata={
            "subject_id": row["subject_id"],
            "hadm_id": row["hadm_id"],
            "admittime": row["admittime"],
            "dischtime": row["dischtime"]
        }
    )
    for _, row in sampled.iterrows()
]

In [ ]:
embedding = HuggingFaceEmbeddings(model_name="./models/all-MiniLM-L6-v2")

In [ ]:
vectorstore = FAISS.from_documents(documents, embedding)
vectorstore.save_local("mimic_vectorstore")

In [ ]:
retriever = vectorstore.as_retriever()

In [ ]:
sampled

In [ ]:
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

llm = Ollama(model="llama3.2:latest")

qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

response = qa_chain.run("How many patients died?")
print(response)